In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
from configparser import ConfigParser

from easydict import EasyDict
from pytorch_lightning import seed_everything

from models.protoconv.data_visualizer import DataVisualizer
from models.protoconv.comet_connector import CometConnector

from utils import plot_html

warnings.simplefilter("ignore")
seed_everything(0)

experiment_id = 'dea01a0186584f4aaffc83c4889418d9'
weights_path = 'fold_0_epoch=23-val_loss_0=0.3930-val_acc_0=0.8472.ckpt'

config = ConfigParser()
config.read('config.ini')
comet_config = EasyDict(config['cometml'])

comet_connector = CometConnector(comet_config.apikey, comet_config.projectname, comet_config.workspace)
comet_connector.set_experiment(experiment_id)
comet_connector.get_model(weights_path)

model = comet_connector.model.cuda()
TEXT=comet_connector.TEXT

data_visualizer = DataVisualizer(model, comet_connector.train_loader, vocab_itos=TEXT.vocab.itos)


ModuleNotFoundError: No module named 'models.protoconv.comet_connector'

In [7]:
plot_html(data_visualizer.visualize_prototypes_as_bold())

In [10]:
text = "The food was _____ and the service was unpleasant. "
tokenized = TEXT.process([TEXT.preprocess(text)]).cuda()
plot_html(data_visualizer.predict(tokenized, true_label=0))


Input,Prototype,Similarity * Importance
unpleasant. <END>,mediocre drinks; bland,1.55 * -0.54 = -0.84
was <unk> and,<START> poor poor,1.12 * -0.62 = -0.70
was <unk> and,not go and,1.27 * -0.17 = -0.22
Input,Prototype,Similarity * Importance
was <unk> and,the great ambiance,1.52 * 0.85 = 1.29
was unpleasant. <END>,cool and cultured;,1.23 * 0.34 = 0.42
was unpleasant. <END>,enjoyed them all.,1.50 * 0.24 = 0.36


In [9]:
text = "The food was tasty and the service was friendly."
tokenized = TEXT.process([TEXT.preprocess(text)]).cuda()
plot_html(data_visualizer.predict(tokenized, true_label=1))

Input,Prototype,Similarity * Importance
friendly. <END>,mediocre drinks; bland,1.55 * -0.54 = -0.84
friendly. <END>,<START> poor poor,1.05 * -0.62 = -0.65
and the service,not go and,1.14 * -0.17 = -0.20
Input,Prototype,Similarity * Importance
was tasty and,the great ambiance,2.51 * 0.85 = 2.12
was friendly. <END>,cool and cultured;,1.23 * 0.34 = 0.42
was friendly. <END>,enjoyed them all.,1.50 * 0.24 = 0.36
